In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext sql

import json

import duckdb
import pandas as pd
import plotly.express as px
import numpy as np
from rich import inspect
from tldextract import extract

from birbnet import config
from birbnet.data_utils import RunDataset

con = duckdb.connect("../data/duck.db")


# for accessing the dataset we'll analyse


## Questions

how to get the most recent scrape for each ID rather than the first one? can I rely on row number with parallelised reading? 


# Connect to the DuckDb database
You can use the `birbnet make-db` command to create a transformed database that this notebook needs.

In [ ]:
dataset = RunDataset("ned_first_run")
conn = dataset.make_duckdb_conn()
%sql conn

In [ ]:
%sql select count(*) from users

In [ ]:
%sql select mean(account_age) from users

In [ ]:
%sql select * from users limit 10

In [ ]:
%sql select username, name, tweet_count, followers_count from users where tweet_count > 20000000 order by tweet_count desc

## Following / Follower counts

In [ ]:
from functools import cached_property


class EdgeDegreeHistogram:
    def __init__(self, edge_type):
        self.edge_type = edge_type

    @cached_property
    def df(self):
        ranges = [
            (1, 10),
            (10, 100),
            (100, 1_000),
            (1_000, 10_000),
            (10_000, 100_000),
            (100_000, 1_000_000_000),
        ]
        counts_df = conn.sql(f"select {self.edge_type}_count from users").df()
        counts = getattr(counts_df, f"{self.edge_type}_count")
        totals = []
        percentages = []
        num_records = len(counts)
        for min_value, max_value in ranges:
            total = ((counts >= min_value) & (counts < max_value)).sum()
            totals.append(total)
            percentages.append(round(100 * total / num_records, 2))
        range_labels = [f"{lower:,}-{upper-1:,}" for lower, upper in ranges]
        range_labels[-1] = "100,000+"
        return pd.DataFrame(
            {self.edge_type: range_labels, "total": totals, "percentage": percentages}
        )

    def plot(self, height=600, **kwargs):
        return px.bar(
            self.df,
            x=self.edge_type,
            y="percentage",
            text=[f"{val:,}" for val in self.df["total"]],
            title=f"Breakdown of users by number of {self.edge_type}",
            height=height,
            **kwargs,
        ).update_layout(title_x=0.5)

In [ ]:
followers_hist = EdgeDegreeHistogram("followers")
followers_hist.plot(height=600, width=800)

In [ ]:
following_hist = EdgeDegreeHistogram("following")
following_hist.plot(height=600, width=800)

In [ ]:
tweet_hist = EdgeDegreeHistogram("tweet")
tweet_hist.plot(height=600, width=800)

In [ ]:
from plotly.subplots import make_subplots

following_hist = EdgeDegreeHistogram("following")
followers_hist = EdgeDegreeHistogram("followers")

fig = make_subplots(rows=1, cols=2)
fig.add_trace(following_hist.plot().data[0], row=1, col=1)
fig.add_trace(followers_hist.plot().data[0], row=1, col=2)
fig.show()

## URL Domain Analysis

In [ ]:
from functools import cache

domain_map = {
    "youtu.be": "youtube.com",
    "amzn.to": "amazon.com",
}

@cache
def get_domain(url, expand=True):
    result = extract(url)
    domain = f"{result.domain}.{result.suffix}".lower()
    if expand:
        domain = domain_map.get(domain, domain)
    return domain


class LinkDomainCounts:
    
    sql = "SELECT urls from users"
    
    def __init__(self, query=None):
        if query is None:
            self.query = self.sql
        else:
            self.query = query

    @cached_property
    def df(self):
        urls_df = conn.sql(self.query).df()
        return (
            urls_df["urls"]
            .explode()
            .dropna()
            .apply(get_domain)
            .value_counts()
            .reset_index()
        )

    def plot(self, limit=10, title="Top Link Domains", **kwargs):
        return px.bar(
            self.df.head(limit).sort_values("count", ascending=True),
            y="urls",
            x="count",
            orientation="h",
            title=title,
            **kwargs,
        )

In [ ]:
domains = LinkDomainCounts()

In [ ]:
domains.plot(limit=20, height=600,width=700)

In [ ]:
domains_10k_followers = LinkDomainCounts(
    query="select urls from users where followers_count >= 10000"
)

In [ ]:
domains_10k_followers.plot(limit=20, height=600,width=700)

In [ ]:
created_dates_df = conn.sql(
f"""
select created_date, count(created_date) as count 
from (
    select date_trunc('day', created_at) as created_date 
    from users
    where created_at >= '2006-01-01'
    order by created_at
) 
group by created_date"""
).df().sort_values("created_date")

px.line(
    created_dates_df,
    x="created_date",
    y="count",
    width=1000,
    height=600,
    title="Number of account creation dates per day",
)